## This notebook is for evaluating the performance of each server from the 3 servers ##
1. MySQL
2. MSSQL (Microsoft SQL Server)
3. PostgreSQL

### Importing the required libraries ###

In [1]:
import time, random

from dotenv import load_dotenv
import os

import pandas as pd, matplotlib.pyplot as plt, seaborn as sns
%matplotlib inline
sns.set()

### Setting the dataframe that will contain the results ###

In [2]:
servers_performance = pd.DataFrame(columns=["MySQL/INSERT", "MySQL/UPDATE", "MSSQL/INSERT", "MSSQL/UPDATE", "PostgreSQL/INSERT", "PostgreSQL/UPDATE"],
                                   index=range(1, 201))
servers_performance = servers_performance.astype(float)
servers_performance.index.name = "StatementID"

In [3]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
196,NaN,NaN,NaN,NaN,NaN,NaN
197,NaN,NaN,NaN,NaN,NaN,NaN
198,NaN,NaN,NaN,NaN,NaN,NaN


### Prepare the data to be inserted ###

In [4]:
# Generate random data
prices = random.sample(range(1000, 10000), 200)
items_in_stock = random.sample(range(100, 1000), 200)
random_ids = random.sample(range(100, 500), 200)
product_ids = [f"P{num}" for num in random_ids]

products_data = pd.DataFrame({"ProductID": product_ids, "Price": prices, "ItemsInStock": items_in_stock},
                             index=range(1, 201))

products_data

,ProductID,Price,ItemsInStock
1,P424,7784,726
2,P388,3415,665
3,P401,4464,116
4,P393,5636,365
5,P232,3890,517
...,...,...,...
196,P218,5063,106
197,P461,8524,522
198,P377,4025,544
199,P480,9908,554


### MySQL Server ###

In [5]:
load_dotenv()
ENV_MYSQL_USER = os.getenv('MYSQL_USER')
ENV_MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')
ENV_MYSQL_HOST = os.getenv('MYSQL_HOST')
ENV_MYSQL_DATABASE = os.getenv('MYSQL_DATABASE')

In [6]:
import mysql.connector as connector

# Connect to the database
connection = connector.connect(user=ENV_MYSQL_USER,
                            password=ENV_MYSQL_PASSWORD,
                            host=ENV_MYSQL_HOST,
                            database=ENV_MYSQL_DATABASE)

In [7]:
# Create a link "cursor" between MySQL and Python
cursor = connection.cursor()

In [8]:
cursor.execute("""DELETE FROM Products;""")

In [9]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 0] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)

In [10]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(100): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 1] = total_time_in_msec
    time.sleep(0.5)

In [11]:
servers_performance[["MySQL/INSERT", "MySQL/UPDATE"]]

,MySQL/INSERT,MySQL/UPDATE
StatementID,,
1,0.000000,7.062912
2,0.000000,1.064777
3,0.000000,0.000000
4,0.000000,1.074791
5,0.504494,0.000000
...,...,...
196,0.000000,NaN
197,2.025843,NaN
198,0.000000,NaN


In [12]:
cursor.close()
connection.close()

### MSSQL ###

In [13]:
load_dotenv()
ENV_MSSQL_SERVER = os.getenv('MSSQL_SERVER')

ENV_MSSQL_DATABASE = os.getenv('MSSQL_DATABASE')

In [14]:
import pyodbc

# Connect to the database using Windows authentication
connection = pyodbc.connect('DRIVER={SQL Server};SERVER='+ENV_MSSQL_SERVER+';DATABASE='+ENV_MYSQL_DATABASE+';Trusted_Connection=yes;')

In [15]:
# Create a link "cursor" between MSSQL and Python
cursor = connection.cursor()

In [16]:
cursor.execute("""DELETE FROM Products;""")

In [17]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 2] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)


In [18]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()

for i in range(200): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 3] = total_time_in_msec
    time.sleep(0.5)

In [19]:
servers_performance[["MSSQL/INSERT", "MSSQL/UPDATE"]]

,MSSQL/INSERT,MSSQL/UPDATE
StatementID,,
1,1.691580,15.952826
2,0.000000,1.073599
3,1.058102,1.560688
4,0.000000,1.560926
5,0.000000,0.000000
...,...,...
196,1.065016,1.467705
197,0.000000,0.000000
198,2.058268,0.000000


In [20]:
cursor.close()
connection.close()

### PostgreSQL ###

In [21]:
load_dotenv()
ENV_POSTGRES_USER = os.getenv('POSTGRES_USER')
ENV_POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
ENV_POSTGRES_HOST = os.getenv('POSTGRES_HOST')
ENV_POSTGRES_DATABASE = os.getenv('POSTGRES_DATABASE')
ENV_POSTGRES_PORT = os.getenv('POSTGRES_PORT')

In [22]:
import psycopg2

# Connect to the database
connection = psycopg2.connect(host=ENV_POSTGRES_HOST,
                                port=ENV_POSTGRES_PORT,
                                database=ENV_POSTGRES_DATABASE,
                                user=ENV_POSTGRES_USER,
                                password=ENV_POSTGRES_PASSWORD)

In [23]:
# Create a link "cursor" between PostgreSQL and Python
cursor = connection.cursor()

In [24]:
cursor.execute("""DELETE FROM Products;""")

In [25]:
# For evaluating INSERT performance
for i in range(200):
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)

    start_time = time.time()
    cursor.execute(add_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 4] = total_time_in_msec

    cursor.execute("""DELETE FROM Products;""")

    time.sleep(0.5)


In [26]:
# For evaluating UPDATE performance
for i in range(200): # Fill the database with data
    product_id, price, items_in_stock = products_data.iloc[i]
    add_record = """INSERT INTO Products VALUES('{}', {}, {});""".format(product_id, price, items_in_stock)
    cursor.execute(add_record)
    connection.commit()
    
for i in range(200): # Update randomly
    new_items_in_stock = 50
    product_id = products_data["ProductID"][i+1]
    update_record = """UPDATE Products SET ItemsInStock = {} WHERE ProductID = '{}';""".format(new_items_in_stock, product_id)

    start_time = time.time()
    cursor.execute(update_record)
    end_time = time.time()
    total_time = end_time - start_time
    total_time_in_msec = total_time*1000
    servers_performance.iloc[i, 5] = total_time_in_msec
    time.sleep(0.5)

In [27]:
servers_performance[["PostgreSQL/INSERT", "PostgreSQL/UPDATE"]]

,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,
1,0.000000,0.000000
2,0.000000,0.000000
3,1.353025,0.000000
4,1.027346,0.000000
5,1.524448,0.000000
...,...,...
196,0.000000,1.526594
197,0.000000,0.000000
198,0.000000,0.000000


In [28]:
cursor.close()
connection.close()

### Statistics ###

In [29]:
servers_performance

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
StatementID,,,,,,
1,0.000000,7.062912,1.691580,15.952826,0.000000,0.000000
2,0.000000,1.064777,0.000000,1.073599,0.000000,0.000000
3,0.000000,0.000000,1.058102,1.560688,1.353025,0.000000
4,0.000000,1.074791,0.000000,1.560926,1.027346,0.000000
5,0.504494,0.000000,0.000000,0.000000,1.524448,0.000000
...,...,...,...,...,...,...
196,0.000000,NaN,1.065016,1.467705,0.000000,1.526594
197,2.025843,NaN,0.000000,0.000000,0.000000,0.000000
198,0.000000,NaN,2.058268,0.000000,0.000000,0.000000


In [30]:
servers_performance.describe()

,MySQL/INSERT,MySQL/UPDATE,MSSQL/INSERT,MSSQL/UPDATE,PostgreSQL/INSERT,PostgreSQL/UPDATE
count,200.000000,100.000000,200.000000,200.000000,200.000000,200.000000
mean,0.469522,0.415208,0.734617,0.738519,0.321351,0.374700
std,0.781348,0.926718,1.257830,2.489154,0.611212,0.591603
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.026034,0.511229,1.199782,0.641346,0.000000,0.633299
max,2.577782,7.062912,8.913040,26.979446,2.283096,2.400637
